# 6. ChainMap

**ChainMap groups multiple dictionaries and other mappings together to create a single object that works like a regular dictionary**. In other words, it takes several mappings and makes them logically appear as one.

ChainMap objects are **updateable views**, which means that changes in any of the chained mappings affect the ChainMap object as a whole. This is because **ChainMap doesn’t merge the input mappings together. It keeps a list of mappings and reimplements common dictionary operations on top of that list.** For example, a key lookup searches the list of mappings successively until it finds the key.

For more details, please visit https://realpython.com/python-chainmap/


## 6.1 Unique keys vs Repeated keys

ChainMap objects can have several dictionaries with either **unique** or **repeated** keys. In either case, ChainMap allows you to treat all your dictionaries as one.


### Unique keys
If across your dictionaries, you only have unique keys, you can access and update the keys as if you were working with a single dictionary.

### Repeated keys
Besides managing your dictionaries as one, you can also take advantage of the internal list of mappings to define some sort of **access priority**. Because of this feature, ChainMap objects are great for handling multiple contexts.

## 6.2 A simple example

Say you’re working on a command-line interface (CLI) application. The application allows the user to use a proxy service for connecting to the Internet. The settings priorities are:

- Command-line options (--proxy, -p)
- Local configuration files in the user’s home directory
- Global proxy configuration

If the user supplies a proxy at the command line, then the application must use that proxy. Otherwise, the application should use the proxy provided in the next configuration object, and so on. This is one of the most common use cases of ChainMap. In this situation, you can do the following:

In [1]:
from collections import ChainMap

In [2]:
cmd_proxy = {}
local_proxy = {"proxy": "proxy.local.com"}
global_proxy = {"proxy": "proxy.global.com"}

my_chain = ChainMap(cmd_proxy, local_proxy, global_proxy)

In [3]:
print(my_chain["proxy"])

proxy.local.com


In the above example, you can notice the order of parameters when we build the ChainMap defines the **priority of the mappings**. In another word, a key lookup of my_chain searches cmd_proxy, then local_proxy, and finally global_proxy, returning the first instance of the key at hand.

## 6.3 More features

ChainMap provides not only all the dictionary features, it also provides some additional features

### 6.3.1 Get list of all mappings

The public attribute **.maps** will return the internal list of mappings

In [4]:
numbers = {"one": 1, "two": 2}
letters = {"a": "A", "b": "B"}

In [7]:
test_chain1 = ChainMap(numbers, letters)

In [8]:
res = test_chain1.maps

In [9]:
print(type(res))

<class 'list'>


In [10]:
print(res)

[{'one': 1, 'two': 2}, {'a': 'A', 'b': 'B'}]


The instance attribute .maps gives you access to the internal list of mappings. **This list is updatable**. You can add and remove mappings manually, iterate through the list, and more.

In below example, we update the chain map via the list returned by .map. Then we search the keys

In [11]:
phones = {"toto": "061034", "titi": "0643103434"}
res.append(phones)

In [12]:
print(test_chain1["toto"])

061034


### 6.3.2 .new_child

With .new_child(), you create a new ChainMap object containing a new map (son) followed by all the maps in the current instance. The map passed as a first argument becomes the first map in the list of maps. If you don’t pass a map, then the method uses an empty dictionary.


In [13]:
dad = {"name": "John", "age": 35}
mom = {"name": "Jane", "age": 31}

test_chain2 = ChainMap(dad, mom)
print(test_chain2)

ChainMap({'name': 'John', 'age': 35}, {'name': 'Jane', 'age': 31})


In [16]:
son = {"name": "titi", "age": 1}
test_chain3 = test_chain2.new_child(son)

print(f"test_chain2 value: {test_chain2}")
print(f"test_chain3 value: {test_chain3}")

test_chain2 value: ChainMap({'name': 'John', 'age': 35}, {'name': 'Jane', 'age': 31})
test_chain3 value: ChainMap({'name': 'titi', 'age': 1}, {'name': 'John', 'age': 35}, {'name': 'Jane', 'age': 31})


In [17]:
# as .map returns a list, we can use for loop
for p in test_chain3.maps:
    print(p)

{'name': 'titi', 'age': 1}
{'name': 'John', 'age': 35}
{'name': 'Jane', 'age': 31}


The **.parents** property returns a new ChainMap objects containing all the maps in the current instance except for the first one. This is useful when you need to skip the first map in a key lookup.

In [18]:
print(test_chain3.parents)

ChainMap({'name': 'John', 'age': 35}, {'name': 'Jane', 'age': 31})


### 6.3.3 Mutating operations

A final feature to highlight in ChainMap is that **mutating operations**, such as
- updating keys,
- adding new keys,
- deleting existing keys,
- popping keys
- clearing the dictionary,
- act on the first mapping in the internal list of mappings

In [19]:
print(test_chain1)

ChainMap({'one': 1, 'two': 2}, {'a': 'A', 'b': 'B'}, {'toto': '061034', 'titi': '0643103434'})


In [20]:
# let's add new key value paires
test_chain1["new_key"]="new_val"

print(test_chain1)

ChainMap({'one': 1, 'two': 2, 'new_key': 'new_val'}, {'a': 'A', 'b': 'B'}, {'toto': '061034', 'titi': '0643103434'})


You can notice the new pair is added to the first mapping

In [22]:
# pop an existing key out of the chainMap
test_chain1.pop("two")

2

In [23]:
print(test_chain1)

ChainMap({'one': 1, 'new_key': 'new_val'}, {'a': 'A', 'b': 'B'}, {'toto': '061034', 'titi': '0643103434'})


So far so good, let's try with a key that are in the **second mappings**

In [24]:
test_chain1.pop("b")

KeyError: "Key not found in the first mapping: 'b'"

Oh, it does not work anymore. Because **mutating operations on a ChainMap object only affect the first mapping in the internal list**. If you want to mutate other mapping, you need to use **.maps** to access the list of mappings.

In [25]:
test_chain1.clear()
print(test_chain1)

ChainMap({}, {'a': 'A', 'b': 'B'}, {'toto': '061034', 'titi': '0643103434'})


The Clear operation can only clear the first mapping too.